# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [96]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import *

## Data Importing

#### - Testing files and paths

In [97]:
test_dir = '../data/AK_Juneau_0.json'
os.path.exists(test_dir)

True

In [98]:
# load one file first to see what type of data you're dealing with and what attributes it has
with open(test_dir) as f:
    data_json = json.load(f)

In [99]:
data_json['data'].keys()

dict_keys(['total', 'count', 'results'])

In [100]:
test_df = pd.json_normalize(data_json['data']['results'])
test_df.head(1)

,last_update_date,tags,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,...,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,products,location.address.coordinate,other_listings
0,2023-09-19T20:52:50Z,"[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",...,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN


#### - Data Overview

In [101]:
# Getting an overview of the data before combining all the files into one dataframe. This is a custom function in `functions_variables.py` --- It takes in a directory path.
overview = json_files_summary('../data')

Error processing file .gitkeep: Expecting value: line 1 column 1 (char 0)
AK_Juneau_0.json processed. 1/251 files processed.
AK_Juneau_1.json processed. 2/251 files processed.
AK_Juneau_2.json processed. 3/251 files processed.
AK_Juneau_3.json processed. 4/251 files processed.
AK_Juneau_4.json processed. 5/251 files processed.
AL_Montgomery_0.json processed. 6/251 files processed.
AL_Montgomery_1.json processed. 7/251 files processed.
AL_Montgomery_2.json processed. 8/251 files processed.
AL_Montgomery_3.json processed. 9/251 files processed.
AL_Montgomery_4.json processed. 10/251 files processed.
AR_LittleRock_0.json processed. 11/251 files processed.
AR_LittleRock_1.json processed. 12/251 files processed.
AR_LittleRock_2.json processed. 13/251 files processed.
AR_LittleRock_3.json processed. 14/251 files processed.
AR_LittleRock_4.json processed. 15/251 files processed.
AZ_Phoenix_0.json processed. 16/251 files processed.
AZ_Phoenix_1.json processed. 17/251 files processed.
AZ_Phoeni

In [102]:
overview
# After reviewing the data with Data Wrangler, I found that some files may be missing the 'results' key, or it is empty.

,file_name,file_path,rows_count,cols_count,cols_name
0,AK_Juneau_0.json,../data\AK_Juneau_0.json,8,64,"[last_update_date, tags, permalink, status, li..."
1,AK_Juneau_1.json,../data\AK_Juneau_1.json,7,63,"[primary_photo, last_update_date, source, tags..."
2,AK_Juneau_2.json,../data\AK_Juneau_2.json,6,63,"[primary_photo, last_update_date, source, tags..."
3,AK_Juneau_3.json,../data\AK_Juneau_3.json,5,63,"[primary_photo, last_update_date, source, tags..."
4,AK_Juneau_4.json,../data\AK_Juneau_4.json,4,63,"[primary_photo, last_update_date, source, tags..."
...,...,...,...,...,...
245,WY_Cheyenne_0.json,../data\WY_Cheyenne_0.json,1,0,[]
246,WY_Cheyenne_1.json,../data\WY_Cheyenne_1.json,1,0,[]
247,WY_Cheyenne_2.json,../data\WY_Cheyenne_2.json,1,0,[]
248,WY_Cheyenne_3.json,../data\WY_Cheyenne_3.json,1,0,[]


In [103]:
overview.shape

(250, 5)

##### - Comparing, Checking files

In [104]:
# 226 -- just chose one of the files that has an empty 'results' key
weird_file_path = overview.iloc[226]['file_path']

with open(weird_file_path) as f:
  weird_file = json.load(f)

weird_file['data'].keys()

dict_keys(['total', 'count', 'results'])

In [105]:
len(weird_file['data']['results']), len(data_json['data']['results']) # Weird vs Normal File

(0, 8)

In [106]:
weird_files = overview[overview['cols_count'] == 0]

weird_files

,file_name,file_path,rows_count,cols_count,cols_name
53,HI_Honolulu_3.json,../data\HI_Honolulu_3.json,1,0,[]
54,HI_Honolulu_4.json,../data\HI_Honolulu_4.json,1,0,[]
100,ME_Augusta_0.json,../data\ME_Augusta_0.json,1,0,[]
101,ME_Augusta_1.json,../data\ME_Augusta_1.json,1,0,[]
102,ME_Augusta_2.json,../data\ME_Augusta_2.json,1,0,[]
103,ME_Augusta_3.json,../data\ME_Augusta_3.json,1,0,[]
104,ME_Augusta_4.json,../data\ME_Augusta_4.json,1,0,[]
120,MS_Jackson_0.json,../data\MS_Jackson_0.json,1,0,[]
121,MS_Jackson_1.json,../data\MS_Jackson_1.json,1,0,[]
122,MS_Jackson_2.json,../data\MS_Jackson_2.json,1,0,[]


In [107]:
# Create a list of files with an empty 'results' key or has other problems
weird = []
extra_weird = []
  
for file in weird_files['file_path']:
  
  with open(file) as f:
    weird_file = json.load(f)
  
  if len(weird_file['data']['results']) == 0:
    print(f'{file} returns empty \'results\'')
    weird.append(file)
  
  else:
    print(f'{file} has another problem...')
    extra_weird.append(file)

../data\HI_Honolulu_3.json returns empty 'results'
../data\HI_Honolulu_4.json returns empty 'results'
../data\ME_Augusta_0.json returns empty 'results'
../data\ME_Augusta_1.json returns empty 'results'
../data\ME_Augusta_2.json returns empty 'results'
../data\ME_Augusta_3.json returns empty 'results'
../data\ME_Augusta_4.json returns empty 'results'
../data\MS_Jackson_0.json returns empty 'results'
../data\MS_Jackson_1.json returns empty 'results'
../data\MS_Jackson_2.json returns empty 'results'
../data\MS_Jackson_3.json returns empty 'results'
../data\MS_Jackson_4.json returns empty 'results'
../data\ND_Bismarck_2.json returns empty 'results'
../data\ND_Bismarck_3.json returns empty 'results'
../data\ND_Bismarck_4.json returns empty 'results'
../data\NH_Concord_3.json returns empty 'results'
../data\NH_Concord_4.json returns empty 'results'
../data\SD_Pierre_0.json returns empty 'results'
../data\SD_Pierre_1.json returns empty 'results'
../data\SD_Pierre_2.json returns empty 'results

In [108]:
print(f'\n{len(weird)} Weird Files...\n{weird}')
print(f'\n{len(extra_weird)} Extra Weird Files... \n{extra_weird}')


32 Weird Files...
['../data\\HI_Honolulu_3.json', '../data\\HI_Honolulu_4.json', '../data\\ME_Augusta_0.json', '../data\\ME_Augusta_1.json', '../data\\ME_Augusta_2.json', '../data\\ME_Augusta_3.json', '../data\\ME_Augusta_4.json', '../data\\MS_Jackson_0.json', '../data\\MS_Jackson_1.json', '../data\\MS_Jackson_2.json', '../data\\MS_Jackson_3.json', '../data\\MS_Jackson_4.json', '../data\\ND_Bismarck_2.json', '../data\\ND_Bismarck_3.json', '../data\\ND_Bismarck_4.json', '../data\\NH_Concord_3.json', '../data\\NH_Concord_4.json', '../data\\SD_Pierre_0.json', '../data\\SD_Pierre_1.json', '../data\\SD_Pierre_2.json', '../data\\SD_Pierre_3.json', '../data\\SD_Pierre_4.json', '../data\\VT_Montpelier_0.json', '../data\\VT_Montpelier_1.json', '../data\\VT_Montpelier_2.json', '../data\\VT_Montpelier_3.json', '../data\\VT_Montpelier_4.json', '../data\\WY_Cheyenne_0.json', '../data\\WY_Cheyenne_1.json', '../data\\WY_Cheyenne_2.json', '../data\\WY_Cheyenne_3.json', '../data\\WY_Cheyenne_4.json']


In [109]:
# Safe to drop the weird files from the overview dataframe
drop = [i for i in weird_files.index]

overview.drop(drop, inplace=True) # Originally 250, 5

In [110]:
overview.shape

(218, 5)

In [111]:
overview # I will use the file_path column to load the data

,file_name,file_path,rows_count,cols_count,cols_name
0,AK_Juneau_0.json,../data\AK_Juneau_0.json,8,64,"[last_update_date, tags, permalink, status, li..."
1,AK_Juneau_1.json,../data\AK_Juneau_1.json,7,63,"[primary_photo, last_update_date, source, tags..."
2,AK_Juneau_2.json,../data\AK_Juneau_2.json,6,63,"[primary_photo, last_update_date, source, tags..."
3,AK_Juneau_3.json,../data\AK_Juneau_3.json,5,63,"[primary_photo, last_update_date, source, tags..."
4,AK_Juneau_4.json,../data\AK_Juneau_4.json,4,63,"[primary_photo, last_update_date, source, tags..."
...,...,...,...,...,...
240,WV_Charleston_0.json,../data\WV_Charleston_0.json,42,61,"[last_update_date, tags, permalink, status, li..."
241,WV_Charleston_1.json,../data\WV_Charleston_1.json,42,61,"[last_update_date, tags, permalink, status, li..."
242,WV_Charleston_2.json,../data\WV_Charleston_2.json,41,61,"[last_update_date, tags, permalink, status, li..."
243,WV_Charleston_3.json,../data\WV_Charleston_3.json,40,61,"[last_update_date, tags, permalink, status, li..."


##### - Resuming Data Import Procedures

In [112]:
# Testing the custom function `read_json` which takes a file path and returns a DataFrame
read_json(test_dir).head(2) # Works!

,last_update_date,tags,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,...,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,products,location.address.coordinate,other_listings
0,2023-09-19T20:52:50Z,"[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...",...,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN
1,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,9424983842,None,...,AK,8477 Thunder Mountain Rd,https://maps.googleapis.com/maps/api/streetvie...,None,Juneau,NaN,NaN,NaN,NaN,NaN


In [113]:
# loop over all files and put them into a dataframe
combine_all = []

for path in overview['file_path']:
  df = read_json(path)
  combine_all.append(df)
  print(f'{path} added to the list. {len(combine_all)}/{overview.shape[0]} processed...')

print(f'\n{len(combine_all)} files loaded into the list of DataFrames.')

../data\AK_Juneau_0.json added to the list. 1/218 processed...
../data\AK_Juneau_1.json added to the list. 2/218 processed...
../data\AK_Juneau_2.json added to the list. 3/218 processed...
../data\AK_Juneau_3.json added to the list. 4/218 processed...
../data\AK_Juneau_4.json added to the list. 5/218 processed...
../data\AL_Montgomery_0.json added to the list. 6/218 processed...
../data\AL_Montgomery_1.json added to the list. 7/218 processed...
../data\AL_Montgomery_2.json added to the list. 8/218 processed...
../data\AL_Montgomery_3.json added to the list. 9/218 processed...
../data\AL_Montgomery_4.json added to the list. 10/218 processed...
../data\AR_LittleRock_0.json added to the list. 11/218 processed...
../data\AR_LittleRock_1.json added to the list. 12/218 processed...
../data\AR_LittleRock_2.json added to the list. 13/218 processed...
../data\AR_LittleRock_3.json added to the list. 14/218 processed...
../data\AR_LittleRock_4.json added to the list. 15/218 processed...
../data\A

In [114]:
housing_dataset = pd.concat(combine_all, ignore_index=True)

housing_dataset.shape

C:\Users\kadm2\AppData\Local\Temp\ipykernel_10816\3315332010.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  housing_dataset = pd.concat(combine_all, ignore_index=True)


(8159, 67)

#### - Saving outputs...

In [115]:
output_path = '../outputs'

# --- !!! Uncomment only when saving. Run once and put the # back. !!! --- #
# housing_dataset.to_csv(os.path.join(output_path, 'housing_dataset.csv'), index=False)

#### - Load

In [116]:
housing_df = pd.read_csv(os.path.join(output_path, 'housing_dataset.csv'))

housing_df.shape

(8159, 67)

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

#### - Quick look at the data and the values in them.

In [117]:
data = housing_df.copy()

In [118]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8159 entries, 0 to 8158
Data columns (total 67 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   last_update_date                       8125 non-null   object 
 1   tags                                   7638 non-null   object 
 2   permalink                              8159 non-null   object 
 3   status                                 8159 non-null   object 
 4   list_date                              7752 non-null   object 
 5   open_houses                            0 non-null      float64
 6   branding                               8159 non-null   object 
 7   list_price                             7721 non-null   float64
 8   property_id                            8159 non-null   int64  
 9   photos                                 7403 non-null   object 
 10  community                              0 non-null      float64
 11  virt

In [119]:
data.describe()

,open_houses,list_price,property_id,community,listing_id,price_reduced_amount,source.plan_id,description.year_built,description.baths_3qtr,description.sold_price,...,location.address.postal_code,location.address.coordinate.lon,location.address.coordinate.lat,location.county.fips_code,primary_photo,source,products,location.address.coordinate,other_listings,location.county
count,0.0,7.721000e+03,8.159000e+03,0.0,7.752000e+03,2.484000e+03,5.000000e+00,7316.000000,566.000000,6.716000e+03,...,8159.000000,7909.000000,7909.000000,7588.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,4.341582e+05,5.755508e+09,NaN,2.957819e+09,2.442704e+04,4.170007e+11,1968.916074,1.247350,4.126050e+05,...,50946.997181,-92.206522,39.009689,28000.253295,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,5.514925e+05,2.687366e+09,NaN,7.541620e+07,7.162396e+04,0.000000e+00,35.096914,0.463482,6.994308e+05,...,29257.110670,15.888886,4.374553,15586.751739,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000e+00,1.003443e+09,NaN,6.052327e+08,1.000000e+02,4.170007e+11,1828.000000,1.000000,3.080000e+02,...,2111.000000,-157.810583,21.277707,1101.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2.090000e+05,3.307743e+09,NaN,2.959499e+09,6.000000e+03,4.170007e+11,1950.000000,1.000000,1.910000e+05,...,25314.000000,-104.971611,35.688084,16001.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,3.250000e+05,6.000993e+09,NaN,2.960836e+09,1.010000e+04,4.170007e+11,1975.000000,1.000000,3.140000e+05,...,50310.000000,-89.333131,39.698210,27123.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,4.999000e+05,8.169927e+09,NaN,2.961805e+09,2.000000e+04,4.170007e+11,1997.000000,1.000000,4.700000e+05,...,78739.000000,-78.617690,41.832266,41047.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,1.250000e+07,9.993679e+09,NaN,2.963230e+09,2.015999e+06,4.170007e+11,2024.000000,3.000000,2.706500e+07,...,99801.000000,-71.006343,58.396178,55025.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
# Quick look at the columns and the data in it -- Using a custom function, it sorts the df by the number of unique values in each column as well. 
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
66,8159,location.county,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
10,8159,community,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
25,8159,description.name,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
37,8159,flags.is_new_construction,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
38,8159,flags.is_for_rent,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


#### Dealing with Nulls

##### - 8159 Nulls : (17 Empty columns)

In [121]:
empties = columns_overview[columns_overview['nulls_count'] == data.shape[0]]

empties.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
66,8159,location.county,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
10,8159,community,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
25,8159,description.name,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
37,8159,flags.is_new_construction,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
38,8159,flags.is_for_rent,float64,0,[nan],"[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


In [122]:
is_empty = [i for i in empties['col_name']]

len(is_empty), is_empty

(17,
 ['location.county',
  'community',
  'description.name',
  'flags.is_new_construction',
  'flags.is_for_rent',
  'flags.is_subdivision',
  'flags.is_contingent',
  'flags.is_pending',
  'flags.is_coming_soon',
  'flags.is_plan',
  'primary_photo',
  'products',
  'other_listings',
  'open_houses',
  'location.address.coordinate',
  'description.baths_1qtr',
  'source'])

In [123]:
# Drop the columns that are empty
data.drop(columns=is_empty, inplace=True)

data.shape[1] # From 67

50

In [124]:
# Drop removed columns from columns_overview
columns_overview.drop(index=empties.index, inplace=True)

columns_overview.shape

(50, 7)

In [125]:
# Base
columns_overview.shape

(50, 7)

In [126]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
49,8154,community.description.name,object,1,"[nan, Woods of Copper Creek]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
14,8154,source.plan_id,float64,1,"[nan, 417000743767.0]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
16,8154,source.spec_id,object,1,"[nan, 365-36546-365750000-0016]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
48,8154,community.advertisers,object,1,"[nan, [{'office': {'hours': 'Monday - Saturday...","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
34,8117,flags.is_foreclosure,object,1,"[nan, True]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


##### - 8154 Nulls : `['community.description.name', 'source.plan_id', 'source.spec_id', 'community.advertisers']`

In [127]:
remove = columns_overview[columns_overview['nulls_count'] == 8154]['col_name'] # Get the columns with 8154 nulls

remove_cols = [i for i in remove]
remove_cols

['community.description.name',
 'source.plan_id',
 'source.spec_id',
 'community.advertisers']

In [128]:
data.drop(columns=remove_cols, inplace=True)

data.shape[1] # From 50

46

In [129]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
32,8117,flags.is_foreclosure,object,1,"[nan, True]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
17,7593,description.baths_3qtr,float64,3,"[nan, 2.0, 1.0, 3.0]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
9,6808,virtual_tours,object,290,"[nan, [{'type': None, 'href': 'https://youtube...","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
25,6732,description.sub_type,object,2,"[nan, condo, townhouse]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"
21,5878,description.baths_half,float64,5,"[nan, 1.0, 2.0, 3.0, 4.0, 5.0]","[nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan]"


##### - 8117 Nulls : `'flags.is_foreclosure'`

In [130]:
# The flags.is_foreclosure may be useful. It may have effect with the house price. Replacing the Nan values with 0 and 1.
data['flags.is_foreclosure'] = data['flags.is_foreclosure'].fillna(0).replace({True: 1})

data['flags.is_foreclosure'].unique()

C:\Users\kadm2\AppData\Local\Temp\ipykernel_10816\318830356.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['flags.is_foreclosure'] = data['flags.is_foreclosure'].fillna(0).replace({True: 1})


array([0, 1], dtype=int64)

##### - 6808 Nulls : `'virtual_tours'`

In [131]:
# Replacing the NaN values with 0, and non-nulls with 1.
data['virtual_tours'] = data['virtual_tours'].notnull().astype(int)

data['virtual_tours'].unique() # We don't really need the values, just the presence of a virtual tour (True/False|0/1) which may have some effect on the price if the owner decided to pay more to have one.

array([0, 1])

##### - 7593 Nulls : `'description.baths_3qtr'`

In [132]:
# Replacing the NaN values with 0
data['description.baths_3qtr'] = data['description.baths_3qtr'].fillna(0.)

data['description.baths_3qtr'].unique()

array([0., 2., 1., 3.])

##### - 6732 Nulls : `'description.sub_type'`

In [133]:
# Calling a property a 'Condo' usually means they're charging more. Replacing Nan with 'No sub_type'
data['description.sub_type'] = data['description.sub_type'].fillna('No sub_type')

data['description.sub_type'].unique()

array(['No sub_type', 'condo', 'townhouse'], dtype=object)

##### - 5878 Nulls : `'description.baths_half'`

In [134]:
# It's possible to have 0 half baths. Replacing the NaN values with 0
data['description.baths_half'] = data['description.baths_half'].fillna(0.)

data['description.baths_half'].unique()

array([0., 1., 2., 3., 4., 5.])

##### - 5675 Nulls : `['flags.is_price_reduced', 'price_reduced_amount']`

In [135]:
# These columns appear to be related.
compare = data[~data[['flags.is_price_reduced', 'price_reduced_amount']].isnull().all(axis=1)]

compare[['flags.is_price_reduced', 'price_reduced_amount']].shape

(2484, 2)

In [136]:
5675 + 2484 == 8159 # All the nulls are in the same rows. Replacing the NaN values with 0 on 'price_reduced_amount' and False on 'flags.is_price_reduced'

True

In [137]:
# Replacing the NaN values with 0
data['price_reduced_amount'] = data['price_reduced_amount'].fillna(0.)

# Replacing the NaN values with False
data['flags.is_price_reduced'] = data['flags.is_price_reduced'].fillna(False)

C:\Users\kadm2\AppData\Local\Temp\ipykernel_10816\4089715487.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['flags.is_price_reduced'] = data['flags.is_price_reduced'].fillna(False)


In [138]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head()

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
26,3711,description.garage,float64,9,"[1.0, nan, 2.0, 3.0, 4.0, 6.0, 5.0, 9.0, 8.0, ...","[1.0, nan, nan, nan, nan]","[1.0, nan, nan, nan, nan]"
27,1899,description.stories,float64,7,"[nan, 1.0, 2.0, 3.0, 10.0, 6.0, 8.0, 4.0]","[nan, nan, nan, nan, nan]","[2.0, nan, nan, nan, nan]"
19,1443,description.sold_price,float64,738,"[nan, 129900.0, 88500.0, 145000.0, 65000.0, 16...","[nan, nan, nan, nan, nan]","[99000.0, 29700.0, 162250.0, 63800.0, 115500.0]"
22,1168,description.lot_sqft,float64,743,"[10454.0, nan, 5875.0, 7476.0, 11761.0, 6534.0...","[10454.0, nan, nan, nan, nan]","[4792.0, 7841.0, 65340.0, nan, 52272.0]"
20,848,description.baths_full,float64,8,"[2.0, nan, 1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]","[2.0, nan, nan, nan, nan]","[1.0, 1.0, 1.0, nan, 2.0]"


##### - 3711 Nulls : `'description.garage'`

In [139]:
# # We can assume Nan means no garage. Replacing the NaN values with 0
data['description.garage'] = data['description.garage'].fillna(0.)

##### - 1899 Nulls : `'description.stories'` -- *Look again*

In [140]:
# 1899 nulls
# 1.0 can be a good guess for the NaN values... unsure...

##### - 1443 Nulls : `'description.sold_price'` -- *Look again*

In [141]:
# 1443 nulls
# Can't guess/assume anything yet... will leave it as is for now.

##### - 1168 Nulls : `'description.lot_sqft'` -- *Look again*

In [142]:
# 1168 nulls
# We can't just replace with 0, may need further investigation.

##### - 848 Nulls : `'description.baths_full'`

In [143]:
# Replacing the NaN values with 0
data['description.baths_full'] = data['description.baths_full'].fillna(0.)

In [144]:
# Re-run the cols_overview function
columns_overview = cols_overview(data)

columns_overview.head(10)

,nulls_count,col_name,col_dtype,nunique,unique,col_data_1,col_data_2
27,1899,description.stories,float64,7,"[nan, 1.0, 2.0, 3.0, 10.0, 6.0, 8.0, 4.0]","[nan, nan, nan, nan, nan]","[2.0, nan, nan, nan, nan]"
19,1443,description.sold_price,float64,738,"[nan, 129900.0, 88500.0, 145000.0, 65000.0, 16...","[nan, nan, nan, nan, nan]","[99000.0, 29700.0, 162250.0, 63800.0, 115500.0]"
22,1168,description.lot_sqft,float64,743,"[10454.0, nan, 5875.0, 7476.0, 11761.0, 6534.0...","[10454.0, nan, nan, nan, nan]","[4792.0, 7841.0, 65340.0, nan, 52272.0]"
16,843,description.year_built,float64,147,"[1963.0, nan, 1969.0, 1920.0, 2002.0, 1998.0, ...","[1963.0, nan, nan, nan, nan]","[1910.0, nan, nan, nan, 1950.0]"
23,836,description.sqft,float64,1119,"[1821.0, nan, 950.0, 3860.0, 1375.0, 1478.0, 1...","[1821.0, nan, nan, nan, nan]","[1214.0, 988.0, 1470.0, nan, 3858.0]"
13,756,primary_photo.href,object,1627,[https://ap.rdcpix.com/07097d34c98a59ebb799688...,[https://ap.rdcpix.com/07097d34c98a59ebb799688...,[https://ap.rdcpix.com/b02cba9b2211b1157e67775...
8,756,photos,object,1627,"[[{'tags': [{'label': 'house_view', 'probabili...","[[{'tags': [{'label': 'house_view', 'probabili...","[[{'tags': [{'label': 'house_view', 'probabili..."
28,655,description.beds,float64,13,"[3.0, nan, 2.0, 5.0, 4.0, 1.0, 6.0, 12.0, 7.0,...","[3.0, nan, nan, nan, nan]","[3.0, 3.0, 3.0, 0.0, 3.0]"
44,571,location.county.fips_code,float64,65,"[nan, 1101.0, 5119.0, 5125.0, 4013.0, 6067.0, ...","[nan, nan, nan, nan, nan]","[54039.0, 54039.0, 54039.0, 54039.0, 54039.0]"
1,521,tags,object,1628,"[['carport', 'community_outdoor_space', 'cul_d...","[['carport', 'community_outdoor_space', 'cul_d...","[['central_air', 'forced_air', 'basement', 'tw..."


##### - 843 Nulls : `'description.year_built'` -- *Look again*

In [145]:
# Needs further investigation

##### - 836 Nulls : `'description.sqft'` -- *Look again*

In [146]:
# Needs further investigation

##### - 756 Nulls : `['primary_photo.href', 'photos']`

In [147]:
# 756 nulls
# These columns appear to be related.
compare = data[~data[['primary_photo.href', 'photos']].isnull().all(axis=1)]

compare[['primary_photo.href', 'photos']].shape

(7403, 2)

In [148]:
756 + 7403 == 8159 # All the nulls are in the same rows. Creating a new column 'has_photos' which will be 0 or 1, and then dropping both columns.

True

In [149]:
data['has_photos'] = data['photos'].notnull().astype(int)

data['has_photos'].unique()

array([1, 0])

In [156]:
data.drop(columns=['primary_photo.href', 'photos'], inplace=True)

##### - 655 Nulls : `'description.beds'` -- *Look again*

In [ ]:
# Needs further investigation

##### - 571 Nulls : `'location.county.fips_code'` -- *Look again*

In [ ]:
# Needs further investigation

##### - 521 Nulls : `'tags'` -- *Gets its own section further down*

##### - 486 Nulls : `'products.brand_name'` -- *Look again*

In [ ]:
# Needs further investigation
# May be connected to source.agents

##### - 438 Nulls : `'list_price'` -- *Look again*

In [ ]:
# Needs further investigation

##### - 407 Nulls : `['source.agents', 'listing_id', 'flags.is_new_listing', 'list_date', 'source.type']` -- *Look again*

In [ ]:
# 'source.agents' may be connected to 'products.brand_name' and/or 'source.type'
# 'flags.is_new_listing' may be connected to 'list_date'

##### - 303 Nulls : `'other_listings.rdc'`


In [157]:
# Will probably drop this column. It's not clear what it is.
data.drop(columns='other_listings.rdc', inplace=True)

##### - 250 Nulls : `['location.address.coordinate.lon', 'location.address.coordinate.lat']`

In [ ]:
# Can be inferred from the rest of the location data.

##### - 179 Nulls : `'description.baths'` -- *Look again*

In [ ]:
# Needs further investigation
# Look into all the columns that have 'baths' in the name.

##### - 34 Nulls : `'last_update_date', 'description.type'` -- *Look again*

In [ ]:
# May drop these nulls, but will check if they're connected to other columns first.
# 'last_update_date' may be connected to 'list_date'
# 'description.type' may be connected to 'description.sub_type'... 'description.type' has the sub_type values. Can probably just merge these 2 columns to avoid redundancy.

##### - 15 Nulls : `'location.address.line'` -- *Look again*

In [ ]:
# Can probably infer 'location.address.coordinate.lon' and 'location.address.coordinate.lat' (250 locations with null) from 'location.address.line' (Only 15 nulls.)
# Need map data. Google Maps API?

##### - 10 Nulls : `'location.county.name'` -- *Look again*

In [ ]:
# Can probably infer from the rest of the location data.

##### - 5 Nulls : `'location.address.city'` -- *Look again*

In [ ]:
# Can probably infer from the rest of the location data.

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [151]:
# OHE categorical variables/ tags here
# tags will have to be done manually

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [152]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [153]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [154]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized

In [155]:
processed_path = '../processed'